In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

enc = OneHotEncoder()
path =  '../web/dataset/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
dev = pd.read_csv(path+'validation.csv')
data = train

In [2]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline


In [3]:
data.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,1,14,fdfae6789b787899f1b875de3ab8b21a,1,u_Vh1OPkFv3q5CFdR,windows_ie,180.107.112.*,80,85,...,60,1,0,5,612599432d200b093719dd1f372f7a30,300,54,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063"
1,0,2,20,372883147cdefbc495fb5082f79dde9b,1,u_VhkRL6dCOTLsq-c,windows_chrome,125.120.199.*,94,95,...,250,1,0,5,8dff45ed862a740986dbe688aafee7e5,249,42,15b749127478946f161a54dc0dad27c8,3476,"10063,10111"
2,0,5,9,2912cb5947e1fad450e2473c2f61ef75,1,u_VhkyORMrD8TP3GL,windows_ie,120.196.64.*,216,217,...,280,0,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,bebefa5efe83beee17a3d245e7c5085b,1458,10063
3,0,2,20,d7b7d61d1e50e1d27d210928d28a9b61,1,u_VhkyLRqVL2a4eWL,windows_ie,219.131.122.*,216,221,...,90,0,0,162,7184c9560e68e977187e67e45a4f3198,238,182,0f951a030abdaedd733ee8d114ce2944,3427,"10077,10111"
4,0,6,9,657d317dc4045bb79c2652abbea07530,1,u_Vh1zOJuQOtTCBgL,windows_chrome,123.190.78.*,40,48,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,34,a3becb967c015e9acb5e72f22b100ef8,3476,"10031,13042,10006,10057,13866,10063,14273,1005..."


In [4]:
data.iloc[0]

click                                            0
weekday                                          1
hour                                            14
bidid             fdfae6789b787899f1b875de3ab8b21a
logtype                                          1
userid                           u_Vh1OPkFv3q5CFdR
useragent                               windows_ie
IP                                   180.107.112.*
region                                          80
city                                            85
adexchange                                       2
domain                        trqRTuToMTNUjM9r5rMi
url               d48a96ab59d7ad741a48e781de44efeb
urlid                                         null
slotid                                   433287550
slotwidth                                      468
slotheight                                      60
slotvisibility                                   1
slotformat                                       0
slotprice                      

In [5]:
os_name = ['windows','android','mac','other','linux','ios']
data["OS"] = float('NaN')
j=0
for i in (os_name):
    data['keypage'][data['keypage']==i] = j
    j +=1

data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="windows"] = 0
data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="android"] = 1
data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="mac"] = 2
data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="other"] = 3
data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="linux"] = 4
data["OS"][data['useragent'].map(lambda x: x.split('_')[0])=="ios"] = 5

In [6]:
browser_name = ['ie','chrome','safari','firefox','other','theworld','maxthon','opera','sogou']
data["Browser"] = float('NaN')
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="ie"] = 0
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="chrome"] = 1
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="safari"] = 2
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="firefox"] = 3
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="other"] = 4
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="theworld"] = 5
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="maxthon"] = 6
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="opera"] = 7
data["Browser"][data['useragent'].map(lambda x: x.split('_')[1])=="sogou"] = 8

In [7]:
data["IPaddress"] = float('NaN')
data["IPaddress"][data["IP"].map(lambda x: x.split('.')[0]).astype(float)<= 255]='4'
data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 239]='3'
data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 223]='2'
data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 191]='1'
data["IPaddress"][data['IP'].map(lambda x: x.split('.')[0]).astype(float)<= 126]='0'
print(data["IPaddress"].value_counts())

0    1866046
2     466982
1     360686
4       2038
3       1986
Name: IPaddress, dtype: int64


In [8]:
a = data['slotwidth']
b = data['slotheight']

data["slotsize"] = float('NaN')
data["slotsize"] = a*b

In [9]:
j=0
for i in (data['keypage'].unique()):
    data['keypage'][data['keypage']==i] = j
    j +=1

In [10]:
j=0
for i in (data['advertiser'].unique()):
    data['advertiser'][data['advertiser']==i] = j
    j +=1

In [11]:
data.iloc[1]

click                                            0
weekday                                          2
hour                                            20
bidid             372883147cdefbc495fb5082f79dde9b
logtype                                          1
userid                           u_VhkRL6dCOTLsq-c
useragent                           windows_chrome
IP                                   125.120.199.*
region                                          94
city                                            95
adexchange                                       2
domain                          trqRTv14MqTR1m58uG
url               313e110c149d24b3236165d30879c595
urlid                                         null
slotid                                  2264242698
slotwidth                                      250
slotheight                                     250
slotvisibility                                   1
slotformat                                       0
slotprice                      

In [12]:
data = data.drop('bidid',1)
data = data.drop('userid',1)
data = data.drop('useragent',1)
data = data.drop('domain',1)
data = data.drop('IP',1)
data = data.drop('url',1)
data = data.drop('urlid',1)
data = data.drop('city',1)
data = data.drop('slotid',1)
data = data.drop('slotwidth',1)
data = data.drop('slotheight',1)
data = data.drop('creative',1)
data = data.drop('usertag',1)
######
data = data.drop('adexchange',1)
data = data.drop('slotvisibility',1)
data = data.drop('slotformat',1)

In [14]:
x_train = np.array(data.drop('click',1))
x_test = np.array(data["click"])

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( x_train, x_test, test_size=0.2, random_state=0)

In [16]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [83]:
from sklearn.tree import DecisionTreeClassifier
my_tree_one = DecisionTreeClassifier()
target = data["click"].values
features_one = data[["weekday","IPaddress","payprice","hour","region","slotprice"]].values
my_tree_one = my_tree_one.fit(features_one,target)

# Look at the importance and score of the included features
print(my_tree_one.feature_importances_)
print(my_tree_one.score(features_one, target))

[ 0.12770554  0.08638804  0.24397613  0.19388204  0.2258      0.04141537
  0.08083289]
0.999595957799


In [84]:
my_prediction = my_tree_one.predict(features_one)
print("DecisionTreeClassifier score on training set: ", rmse(target, my_prediction))

DecisionTreeClassifier score on training set:  0.0201008010109


In [87]:
#Control overfitting by setting "max_depth" to 10 and "min_samples_split" to 5 : my_tree_two ######prevent overfitting
max_depth = 10
min_samples_split =5
my_tree_two = DecisionTreeClassifier(max_depth = 10, min_samples_split = 5, random_state = 0)
my_tree_two = my_tree_two.fit(features_one, target)

#Print the score of the new decison tree
print(my_tree_two.feature_importances_)

print(my_tree_two.score(features_one, target))

[ 0.12068022  0.04487074  0.34425469  0.15876472  0.11932194  0.03291315
  0.17919456]
0.999248259097


In [88]:
my_prediction2 = my_tree_two.predict(features_one)
print("DecisionTreeClassifier score on training set2: ", rmse(target, my_prediction2))

DecisionTreeClassifier score on training set:  0.0274178938457


In [18]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
regr = RandomForestRegressor(
    n_jobs= 1,
    n_estimators= 100,
    max_features= 'auto',
    max_depth= 2,
    min_samples_split=2,
    min_samples_leaf= 1)

In [39]:
regr.fit(X_train[:100000], y_train[:100000])
# Run prediction on training set to get a rough idea of how well it does.
y_pred = regr.predict(X_test[:100000])
print("RandomForestRegressor score on training set: ", rmse(y_train[:100000], y_pred))


# Run prediction on the Kaggle test set.
y_pred_2 = regr.predict(X_test[:10000])
print("RandomForestRegressor score on vali set: ", rmse(y_test[:10000], y_pred_2))

RandomForestRegressor score on training set:  0.0249270505246
RandomForestRegressor score on vali set:  0.0300238230321
